# FRQ 2
> Relationship of FRQ 2 to Project Based learning.
- title: FRQ2 - Classes
- toc: true
- badges: false
- image: /images/frqs.png
- categories: [1.B]
- tags: [api]
- type: ap
- week: 13

## Classes Background
> All Classes FRQs relate well to POJOs in PBL.  The 2019 FRQ #2 describes a problem of a Step Tracker.  The task is to write a StepTracker class.   Included is the constructor and any required instance variables and
methods.
- Constructor: StepTracker tr = new StepTracker(10000);
- Get Methods with algorithms: tr.activeDays(); tr.averageSteps(); 
- Set Methods: tr.addDailySteps(9000);

### Assumptions to PBL this Project
> This is a fun problem and could be expanded upon with PBL.  The StepTracker could be a POJO in Java. As a PBL concept, StepTracking is something that is done by an individual in relation to their personal fitness. What designs or objects would you need for StepTracking...

- Backend Assumptions:
    - This problem has one-to-many relationship between "Person" object to "Day" objects.  
    - Each "Day" is were the "steps" attribute would need to be kept.  
    - Active days implies a "Person" object needs a "goal_steps" to be maintained against the "steps" for a "Day".
    - Person Class. Contact and biographical information, like age/height/weight could be important to this application.  Such information could be used to help calculate goals steps.
    - Day Class.  Steps is key element.  Perhaps application could include run or walking time.  Application could be extended to have things like calorie intake, sleep hours, mood ratio, or other factors that could help in determining goal versus accomplishments.
    - Other Class(es).  A Day Class implies a Month or Year relationship.  Investigation should be made to determine best Java Modeling and Algorithms available.

- Frontend Assumptions:
    - A persons steps need to be recorded by a person in the Frontend
    - A person would need to view or perform analytics on self, perhaps a daily step chart.
    - A person might want to know about how they met or missed goals versus food calorie intake, hours slept, mood ratio.

## Starting the Project
> To study Classes, which is the intent of FRQ2, A PBL project step tracking seems to begin with a "Person" Class.  This may be a deviation from FRQ 2 simplicity, but it is higher level way to think of this problem.

### Person POJO
> This POJO is setup as an @Entity and use Lambok extensively to reduce code required by Developer(s).  This POJO can be used for many Projects!!!

In [ ]:
package com.nighthawk.spring_portfolio.mvc.person;

import org.springframework.format.annotation.DateTimeFormat;
import lombok.*;

import javax.persistence.*;
import javax.validation.constraints.*;
import java.time.LocalDate;
import java.time.Period;
import java.time.ZoneId;
import java.util.*;

/*
Person is a POJO, Plain Old Java Object.
First set of annotations add functionality to POJO
--- @Setter @Getter @ToString @NoArgsConstructor @RequiredArgsConstructor
The last annotation connect to database
--- @Entity
 */
@Data
@NoArgsConstructor
@AllArgsConstructor
@Entity
public class Person {
    // automatic unique identifier for Person record
    @Id
    @GeneratedValue(strategy = GenerationType.AUTO)
    private Long id;

    // email, password, roles are key to login and authentication
    @NotEmpty
    @Size(min=5)
    @Column(unique=true)
    @Email
    private String email;

    @NotEmpty
    private String password;

    // @NonNull: Places this in @RequiredArgsConstructor
    @NonNull
    @Size(min = 2, max = 30, message = "Name (2 to 30 chars)")
    private String name;

    @DateTimeFormat(pattern = "yyyy-MM-dd")
    private Date dob;

    // Initializer used when setting database from an API
    public Person(String email, String password, String name, Date dob) {
        this.email = email;
        this.password = password;
        this.name = name;
        this.dob = dob;
    }

    // A custom getter to return age from dob calculation
    public int getAge() {
        if (this.dob != null) {
            LocalDate birthDay = this.dob.toInstant().atZone(ZoneId.systemDefault()).toLocalDate();
            return Period.between(birthDay, LocalDate.now()).getYears(); }
        return -1;
    }

}

## Next Steps
> Building the Spring Java Persistent API (JPA) and the RestController API enable this or any project to get off the ground.   The JPA works with the database and RestController enables access by the Frontend developers.

### Spring Data JPA
> The "likeability" of Java for me and for the Computer Science industry is the ease of database management.  By defining a POJO and getting all the database management under the hood is very cool!
- BTW, there is a task to change to AWS in the future, this can be done by changing application properties and and POM dependencies, with little or no change to code.
- Review code, there are Long interfaces and native SQL options in extending interfaces into the JpaRepository.

In [ ]:
package com.nighthawk.spring_portfolio.mvc.person;

import org.springframework.data.jpa.repository.JpaRepository;
import org.springframework.data.jpa.repository.Query;

import java.util.List;

/*
Extends the JpaRepository interface from Spring Data JPA.
-- Java Persistent API (JPA) - Hibernate: map, store, update and retrieve database
-- JpaRepository defines standard CRUD methods
-- Via JPA the developer can retrieve database from relational databases to Java objects and vice versa.
 */
public interface PersonJpaRepository extends JpaRepository<Person, Long> {
    Person findByEmail(String email);

    List<Person> findAllByOrderByNameAsc();

    // JPA query, findBy does JPA magic with "Name", "Containing", "Or", "Email", "IgnoreCase"
    List<Person> findByNameContainingIgnoreCaseOrEmailContainingIgnoreCase(String name, String email);
    /* Custom JPA query articles, there are articles that show custom SQL as well
       https://springframework.guru/spring-data-jpa-query/
       https://docs.spring.io/spring-data/jpa/docs/current/reference/html/#jpa.query-methods
     */

    // Custom JPA query
    @Query(
            value = "SELECT * FROM Person p WHERE p.name LIKE ?1 or p.email LIKE ?1",
            nativeQuery = true)
    List<Person> findByLikeTermNative(String term);
    /*
        https://www.baeldung.com/spring-data-jpa-query
     */
}

### Sprint RestController
> This is interface between Backend implementation and the world.  Get, Post, Delete mappings are shown.  In this code, you will see many interactions with the POJO as well as interactions with the JPA.

In [ ]:
package com.nighthawk.spring_portfolio.mvc.person;

import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.http.HttpStatus;
import org.springframework.http.MediaType;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

import java.util.*;
import java.text.SimpleDateFormat;

@RestController
@RequestMapping("/api/person")
public class PersonApiController {
    /*
    #### RESTful API ####
    Resource: https://spring.io/guides/gs/rest-service/
    */

    // Autowired enables Control to connect POJO Object through JPA
    @Autowired
    private PersonJpaRepository repository;

    /*
    GET List of People
     */
    @GetMapping("/")
    public ResponseEntity<List<Person>> getPeople() {
        return new ResponseEntity<>( repository.findAllByOrderByNameAsc(), HttpStatus.OK);
    }

    /*
    GET individual Person using ID
     */
    @GetMapping("/{id}")
    public ResponseEntity<Person> getPerson(@PathVariable long id) {
        Optional<Person> optional = repository.findById(id);
        if (optional.isPresent()) {  // Good ID
            Person person = optional.get();  // value from findByID
            return new ResponseEntity<>(person, HttpStatus.OK);  // OK HTTP response: status code, headers, and body
        }
        // Bad ID
        return new ResponseEntity<>(HttpStatus.BAD_REQUEST);       
    }

    /*
    DELETE individual Person using ID
     */
    @DeleteMapping("/delete/{id}")
    public ResponseEntity<Person> deletePerson(@PathVariable long id) {
        Optional<Person> optional = repository.findById(id);
        if (optional.isPresent()) {  // Good ID
            Person person = optional.get();  // value from findByID
            repository.deleteById(id);  // value from findByID
            return new ResponseEntity<>(person, HttpStatus.OK);  // OK HTTP response: status code, headers, and body
        }
        // Bad ID
        return new ResponseEntity<>(HttpStatus.BAD_REQUEST); 
    }

    /*
    POST Aa record by Requesting Parameters from URI
     */
    @PostMapping( "/post")
    public ResponseEntity<Object> postPerson(@RequestParam("email") String email,
                                             @RequestParam("password") String password,
                                             @RequestParam("name") String name,
                                             @RequestParam("dob") String dobString) {
        Date dob;
        try {
            dob = new SimpleDateFormat("MM-dd-yyyy").parse(dobString);
        } catch (Exception e) {
            return new ResponseEntity<>(dobString +" error; try MM-dd-yyyy", HttpStatus.BAD_REQUEST);
        }
        // A person object WITHOUT ID will create a new record with default roles as student
        Person person = new Person(email, password, name, dob);
        repository.save(person);
        return new ResponseEntity<>(email +" is created successfully", HttpStatus.CREATED);
    }

    /*
    The personSearch API looks across database for partial match to term (k,v) passed by RequestEntity body
     */
    @PostMapping(value = "/search", produces = MediaType.APPLICATION_JSON_VALUE)
    public ResponseEntity<Object> personSearch(@RequestBody final Map<String,String> map) {
        // extract term from RequestEntity
        String term = (String) map.get("term");

        // JPA query to filter on term
        List<Person> list = repository.findByNameContainingIgnoreCaseOrEmailContainingIgnoreCase(term, term);

        // return resulting list and status, error checking should be added
        return new ResponseEntity<>(list, HttpStatus.OK);
    }
}


### Test RESTController
> Recommendation are to test a RESTController using Curl or Postman. 
- Curl is command line oriented.  It is not discussed but could be used for automation.  For instance, you could use it with Bash to load test data. 
- ***PostMan*** is more visual.  Postman is very "browser like".  Test outputs will show Postman visuals and discuss highlight different access methods.  These methods are implementations available in RestController.

> Adding record using Params and GET.
![add](images/postman_add.png)

> Listing records using GET.
![add](images/postman_list.png)

> Searching records using Body raw and JSON using POST.  This is preferred method to secure transmission
![add](images/postman_search.png)

> Delete record using DELETE.
![add](images/postman_delete.png)




### Collection, List, ArrayList, HashMap
> The usage of Collection, List, ArrayList, HashMap enables a POJO to reference other POJOs.  Often in Java you will see these different names, here is brief definition...
- A Collection is an interface that defines the highest-level of shared collection behavior, and extends Iterable which allows usage of ForEach loop.
- A List is an interface that defines the highest-level of shared List behavior. The type List has in its contract that all elements have an order, that the order will not change except through intended manipulation.  Key interfaces: A List allows get() the n'th element;  A List defines add() to put new elements at the end. 
- ArrayList is an implementation of List.  ArrayList should not be used in an API. Its usage should only be used in implementation detail.
- HashMap demands key be unique.  Data is formed in Key/Value pairs.

## Hacks
> Start with Person POJO.  
- Build other attributes into POJO that where mentioned in project dialog at the beginning of this blog.  Make Issues to describe plan and work.
- Start thinking about building other POJOs (ie Day).   In these POJOs you will have a relationship (see below).  For instance, if you define Day POJO, the POJO Person and the POJO Day will contain relationships.  List is the interface and ArrayList is the implementation.  Start now, we will revisit this when we talk about the ArrayLists FRQ. 
- [Read this article for Background](https://vladmihalcea.com/the-best-way-to-map-a-onetomany-association-with-jpa-and-hibernate/).

In [ ]:
/*
"1 to Many" and "Many to 1"
- 1 Person to Many Day(s) Relationship (Forward)
- Many Days to 1 Person Relationship (Reverse)
- symbol "-|" means the Day to Person must contain a result
- symbol "O<-" means there are zero to many Day(s) were data is collected
-----------              -----------
| Person  | -|-----O<- | Day     |  
-----------              -----------
*/

// Relationship shown, data is incomplete
@Entity
public class Person {
    // Person data

    // relationship
    @OneToMany(
        cascade = CascadeType.ALL,
        orphanRemoval = true
    )
    private List<Day> days = new ArrayList<>();
}

// Relationship shown, data is incomplete
@Entity
public class Day {
    // attributes, how would you track days.  perhaps capture date and have many method to calculate this data
    private int year;
    private int day;
    private int steps;
 
    // relationship
    @ManyToOne(fetch = FetchType.LAZY)
    private Person person;
}